In [20]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import os

ROOT.gErrorIgnoreLevel = ROOT.kError

# ─────────────────────────────────────────────
# 0) Data / MC 파일 리스트 (2024)
# ─────────────────────────────────────────────

data_files = [
    # EGamma0
    "EGamma0_Data_2024C.root",
    "EGamma0_Data_2024D.root",
    "EGamma0_Data_2024E.root",
    "EGamma0_Data_2024F.root",
    "EGamma0_Data_2024G.root",
    "EGamma0_Data_2024H.root",
    "EGamma0_Data_2024I.root",
    "EGamma0_v2_Data_2024I.root",

    # EGamma1
    "EGamma1_Data_2024C.root",
    "EGamma1_Data_2024D.root",
    "EGamma1_Data_2024E.root",
    "EGamma1_Data_2024F.root",
    "EGamma1_Data_2024G.root",
    "EGamma1_Data_2024H.root",
    "EGamma1_Data_2024I.root",
    "EGamma1_v2_Data_2024I.root",

    # Muon0
    "Muon0_Data_2024C.root",
    "Muon0_Data_2024D.root",
    "Muon0_Data_2024E.root",
    "Muon0_Data_2024F.root",
    "Muon0_Data_2024G.root",
    "Muon0_Data_2024H.root",
    "Muon0_Data_2024I.root",
    "Muon0_v2_Data_2024I.root",

    # Muon1
    "Muon1_Data_2024C.root",
    "Muon1_Data_2024D.root",
    "Muon1_Data_2024E.root",
    "Muon1_Data_2024F.root",
    "Muon1_Data_2024G.root",
    "Muon1_Data_2024H.root",
    "Muon1_Data_2024I.root",
    "Muon1_v2_Data_2024I.root",
]

mc_files = [
    "DY2E_2J_50_MC_2024.root",
    "DY2M_2J_50_MC_2024.root",
    # "DY2T_2J_50_MC_2024.root",

    "s-channel_antitop_MC_2024.root",
    "s-channel_top_MC_2024.root",
    "ST_tW_antitop_di_MC_2024.root",
    "ST_tW_antitop_semi_MC_2024.root",
    "ST_tW_top_di_MC_2024.root",
    "ST_tW_top_semi_MC_2024.root",
    "t-channel_antitop_MC_2024.root",
    "t-channel_top_MC_2024.root",

    "Ttbar_di_MC_2024.root",
    "Ttbar_semi_MC_2024.root",

    "WJets_1J_MC_2024.root",
    "WJets_2J_MC_2024.root",
    "WJets_3J_MC_2024.root",
    "WJets_4J_MC_2024.root",

    "WW_MC_2024.root",
    "WWW_4F_MC_2024.root",
    "WZ_MC_2024.root",
    "WZZ_5F_MC_2024.root",
    "ZZ_22_MC_2024.root",
    "ZZ_4_MC_2024.root",
    "ZZ_semi_MC_2024.root",
    "ZZZ_5F_MC_2024.root",
]

# ─────────────────────────────────────────────
# 0.5) run(era)별 루미 (fb^-1)
# ─────────────────────────────────────────────

era_lumi_fb = {
    "2024C": 7.24,
    "2024D": 7.96,
    "2024E": 11.32,
    "2024F": 27.76,
    "2024G": 37.77,
    "2024H": 5.44,
    "2024I": 11.47,
}

def get_era_from_filename(fn):
    for era in era_lumi_fb.keys():
        if era in fn:
            return era
    return None

used_eras = set()
for fn in data_files:
    era = get_era_from_filename(fn)
    if era is not None:
        used_eras.add(era)

if not used_eras:
    raise RuntimeError("data_files에서 era를 하나도 못 찾았음. 파일 이름/era_lumi_fb를 확인해줘.")

luminosity_fb = sum(era_lumi_fb[era] for era in used_eras)
luminosity_pb = luminosity_fb * 1000.0  # fb^-1 → pb^-1

print("[INFO] 사용된 era:", used_eras)
print("[INFO] 총 루미 (fb^-1):", luminosity_fb)
print("[INFO] 총 루미 (pb^-1):", luminosity_pb)

# ─────────────────────────────────────────────
# 0.6) MC cross section (pb) — BR 반영
# ─────────────────────────────────────────────

cross_sections_pb = {
    "DY2E_2J_50_MC_2024.root":      6639,
    "DY2M_2J_50_MC_2024.root":      6662,
    # "DY2T_2J_50_MC_2024.root":    ...,

    "s-channel_antitop_MC_2024.root":    1.43,
    "s-channel_top_MC_2024.root":        2.278,

    "ST_tW_antitop_di_MC_2024.root":     36.05 * 0.10621,
    "ST_tW_antitop_semi_MC_2024.root":   36.05 * 0.43938,
    "ST_tW_top_di_MC_2024.root":         35.99 * 0.10621,
    "ST_tW_top_semi_MC_2024.root":       35.99 * 0.43938,

    "t-channel_antitop_MC_2024.root":    23.34,
    "t-channel_top_MC_2024.root":        38.6,

    "Ttbar_di_MC_2024.root":             762.1 * 0.10621081,
    "Ttbar_semi_MC_2024.root":           762.1 * 0.43937838,

    "WJets_1J_MC_2024.root":             9141,
    "WJets_2J_MC_2024.root":             2931,
    "WJets_3J_MC_2024.root":             864.6,
    "WJets_4J_MC_2024.root":             417.8,

    "WW_MC_2024.root":                   11.79,
    "WWW_4F_MC_2024.root":               0.2328,
    "WZ_MC_2024.root":                   4.924,
    "WZZ_5F_MC_2024.root":               0.06206,
    "ZZ_22_MC_2024.root":                1.031,
    "ZZ_4_MC_2024.root":                 1.39,
    "ZZ_semi_MC_2024.root":              6.788,
    "ZZZ_5F_MC_2024.root":               0.01591,
}

# ─────────────────────────────────────────────
# 0.7) 샘플 → 물리 그룹 매핑
# ─────────────────────────────────────────────
sample_to_group = {
    # DY
    "DY2E_2J_50_MC_2024.root": "DY",
    "DY2M_2J_50_MC_2024.root": "DY",
    # "DY2T_2J_50_MC_2024.root": "DY",

    # single top s-channel
    "s-channel_antitop_MC_2024.root": "single top s-channel",
    "s-channel_top_MC_2024.root":     "single top s-channel",

    # single top tW
    "ST_tW_antitop_di_MC_2024.root":   "single top tW",
    "ST_tW_antitop_semi_MC_2024.root": "single top tW",
    "ST_tW_top_di_MC_2024.root":       "single top tW",
    "ST_tW_top_semi_MC_2024.root":     "single top tW",

    # single top t-channel
    "t-channel_antitop_MC_2024.root":  "single top t-channel",
    "t-channel_top_MC_2024.root":      "single top t-channel",

    # ttbar
    "Ttbar_di_MC_2024.root":   "Ttbar",
    "Ttbar_semi_MC_2024.root": "Ttbar",

    # W+jets
    "WJets_1J_MC_2024.root":   "W+jets",
    "WJets_2J_MC_2024.root":   "W+jets",
    "WJets_3J_MC_2024.root":   "W+jets",
    "WJets_4J_MC_2024.root":   "W+jets",

    # diboson
    "WW_MC_2024.root":         "Diboson",
    "WZ_MC_2024.root":         "Diboson",
    "ZZ_22_MC_2024.root":      "Diboson",
    "ZZ_4_MC_2024.root":       "Diboson",
    "ZZ_semi_MC_2024.root":    "Diboson",

    # triboson
    "WWW_4F_MC_2024.root":     "Triboson",
    "WZZ_5F_MC_2024.root":     "Triboson",
    "ZZZ_5F_MC_2024.root":     "Triboson",
}

# 스택에서의 그룹 순서 (맨 마지막이 맨 위)
group_order = [
    "single top s-channel",  # ★ 항상 맨 위
    "DY",
    "W+jets",
    "Ttbar",
    "single top t-channel",
    "single top tW",
    "Diboson",
    "Triboson",
]

# 그룹별 색깔 (빨주노초파 느낌, s-channel = 빨강)
group_colors = {
    # 가장 많이 나오는 메이저 배경 2개 → 채도 높은 색
    "W+jets":              "#2563EB",  # 진한 파란색 (메인 1)
    "Ttbar":               "#F97316",  # 진한 오렌지 (메인 2)

    # 그 다음 규모 배경들 → 중간 채도
    "DY":                  "#14B8A6",  # 청록/티얼 그린
    "single top t-channel":"#84CC16",  # 옐로우-그린

    # single top tW → 다른 계열로 확실히 구분
    "single top tW":       "#8B5CF6",  # 보라

    # 레어 배경들 → 채도/명도 낮춰서 조용하게
    "Diboson":             "#64748B",  # 블루-그레이
    "Triboson":            "#4B5563",  # 더 어두운 그레이

    # 신호: s-channel (항상 눈에 띄게)
    "single top s-channel":"#DC2626",  # 강한 레드
}


out_dir = "compare_plots_mpl_stack_v3_2024_log"
os.makedirs(out_dir, exist_ok=True)

# ─────────────────────────────────────────────
# 1) 히스토그램 이름 추출
# ─────────────────────────────────────────────
def get_hist_names(fn):
    f = ROOT.TFile.Open(fn)
    if not f or f.IsZombie():
        raise RuntimeError("파일을 열 수 없음: %s" % fn)
    f.cd("plots")
    names = [k.GetName() for k in ROOT.gDirectory.GetListOfKeys()]
    f.Close()
    return names

hist_names = get_hist_names(mc_files[0])

# mu_*, e_*, combine_* 만
prefixes = ("mu", "e", "combine")
hist_names = [h for h in hist_names if any(h.startswith(f"{p}_") for p in prefixes)]

# ─────────────────────────────────────────────
# 2) 히스토그램 루프
# ─────────────────────────────────────────────
for hname in hist_names:
    print("▶", hname)

    # --- Data 합치기 ---
    data_h = None
    for fn in data_files:
        if not os.path.isfile(fn):
            continue
        f = ROOT.TFile.Open(fn)
        if not f or f.IsZombie():
            f.Close()
            continue
        h = f.Get(f"plots/{hname}")
        if h:
            if data_h is None:
                data_h = h.Clone("data_tmp")
                data_h.SetDirectory(0)
            else:
                data_h.Add(h)
        f.Close()

    if data_h is None:
        print("   ✖ No data, skipping")
        continue

    # --- MC: 샘플 → 그룹으로 합산 ---
    group_sums = {}  # group_name -> np.array

    for fn in mc_files:
        if not os.path.isfile(fn):
            continue

        f = ROOT.TFile.Open(fn)
        if not f or f.IsZombie():
            f.Close()
            continue

        h = f.Get(f"plots/{hname}")
        if not h:
            f.Close()
            continue

        cnt   = f.Get("plots/count")
        n_evt = cnt.GetSumOfWeights() if cnt else 0.0
        if n_evt <= 0:
            f.Close()
            continue

        xsec  = cross_sections_pb.get(fn, 1.0)
        scale = (xsec * luminosity_pb) / n_evt

        tmp = h.Clone("tmp")
        tmp.SetDirectory(0)
        tmp.Sumw2()
        tmp.Scale(scale)

        nb  = tmp.GetNbinsX()
        arr = np.array([tmp.GetBinContent(i) for i in range(1, nb + 1)])

        group = sample_to_group.get(fn, "other")
        if group not in group_sums:
            group_sums[group] = arr
        else:
            group_sums[group] += arr

        f.Close()

    # stack 순서/색깔 준비
    mc_counts_list = []
    mc_labels      = []
    mc_colors      = []

    for g in group_order:
        if g in group_sums:
            mc_counts_list.append(group_sums[g])
            mc_labels.append(g)
            mc_colors.append(group_colors.get(g, None))

    if not mc_counts_list:
        print("   ✖ No MC, skipping")
        continue

    if data_h.GetDimension() != 1:
        print("   ✖ Skipping non-1D histogram")
        continue

    # NumPy 변환
    nb = data_h.GetNbinsX()
    edges = np.array(
        [data_h.GetBinLowEdge(i) for i in range(1, nb + 1)]
        + [data_h.GetBinLowEdge(nb) + data_h.GetBinWidth(nb)]
    )
    dat_counts  = np.array([data_h.GetBinContent(i) for i in range(1, nb + 1)])
    dat_err     = np.sqrt(dat_counts)
    bin_centers = 0.5 * (edges[:-1] + edges[1:])

    # ─ Plot ─
    plt.style.use(hep.style.CMS)
    fig, (ax, axr) = plt.subplots(
        nrows=2,
        sharex=True,
        gridspec_kw={"height_ratios": [3, 1], "hspace": 0.05},
        figsize=(10, 8),
        dpi=150,
    )

    # MC 스택 (그룹 단위, 색 지정)
    hep.histplot(
        mc_counts_list,
        bins=edges,
        stack=True,
        histtype="fill",
        label=mc_labels,
        color=mc_colors,
        ax=ax,
    )

    # Data
    ax.errorbar(
        bin_centers,
        dat_counts,
        yerr=dat_err,
        fmt="o",
        color="black",
        label="Data",
    )
    ax.set_yscale("log")
    ax.set_ylabel("Events / bin")

    # ★ 13.6 TeV 로 표기
    hep.cms.label(
        "Private Work",
        data=True,
        lumi=round(luminosity_fb, 2),
        year=2024,
        com=13.6,            # <── 여기 때문에 "13.6 TeV" 로 나옴
        ax=ax,
    )

    # ── 여기서 Legend 순서 재정렬 ─────────────────────────
    # 원하는 legend 순서:
    legend_order = [
        "single top t-channel",
        "single top tW",
        "Ttbar",
        "W+jets",
        "DY",
        "Diboson",
        "Triboson",
        "single top s-channel",
        "Data",
    ]

    handles, labels = ax.get_legend_handles_labels()
    label2handle = {lab: h for h, lab in zip(handles, labels)}

    ordered_handles = [label2handle[lab] for lab in legend_order if lab in label2handle]
    ordered_labels  = [lab for lab in legend_order if lab in label2handle]

    ax.legend(
        ordered_handles,
        ordered_labels,
        loc="upper right",
        prop={"size": 12},
        handletextpad=0.2,
        labelspacing=0.2,
        columnspacing=0.5,
    )

    # Ratio
    mc_sum = np.sum(mc_counts_list, axis=0)
    mask   = mc_sum > 0
    ratio     = np.divide(dat_counts, mc_sum,
                          out=np.zeros_like(dat_counts), where=mask)
    ratio_err = np.divide(dat_err,    mc_sum,
                          out=np.zeros_like(dat_err),    where=mask)

    axr.errorbar(
        bin_centers[mask],
        ratio[mask],
        yerr=ratio_err[mask],
        fmt="o",
        color="black",
    )
    axr.axhline(1.0, color="gray", linestyle="--", linewidth=1)
    axr.set_ylabel("Data/MC")
    axr.set_xlabel(hname)
    axr.set_ylim(0.5, 1.5)

    out_path = os.path.join(out_dir, f"stack_{hname}.png")
    fig.savefig(out_path)
    plt.close(fig)

print("✅ All stacked plots saved in", out_dir)


[INFO] 사용된 era: {'2024E', '2024G', '2024D', '2024H', '2024F', '2024I', '2024C'}
[INFO] 총 루미 (fb^-1): 108.96
[INFO] 총 루미 (pb^-1): 108960.0
▶ e_2_jet_1_btag_MET
▶ e_2_jet_1_btag_lep1pt
▶ e_2_jet_1_btag_lep1pt_None
▶ e_2_jet_1_btag_lep1eta
▶ e_2_jet_1_btag_jet1eta
▶ e_2_jet_1_btag_jet2eta
▶ e_2_jet_1_btag_jet3eta


/tmp/ipykernel_29479/322654193.py:391: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  ax.set_yscale("log")


▶ e_2_jet_1_btag_jet1pt
▶ e_2_jet_1_btag_jet2pt
▶ e_2_jet_1_btag_jet3pt
▶ mu_2_jet_1_btag_MET
▶ mu_2_jet_1_btag_lep1pt
▶ mu_2_jet_1_btag_lep1pt_None
▶ mu_2_jet_1_btag_lep1eta
▶ mu_2_jet_1_btag_jet1eta
▶ mu_2_jet_1_btag_jet2eta
▶ mu_2_jet_1_btag_jet3eta
▶ mu_2_jet_1_btag_jet1pt
▶ mu_2_jet_1_btag_jet2pt
▶ mu_2_jet_1_btag_jet3pt
▶ combine_2_jet_1_btag_MET
▶ combine_2_jet_1_btag_lep1pt
▶ combine_2_jet_1_btag_lep1pt_None
▶ combine_2_jet_1_btag_lep1eta
▶ combine_2_jet_1_btag_jet1eta
▶ combine_2_jet_1_btag_jet2eta
▶ combine_2_jet_1_btag_jet3eta
▶ combine_2_jet_1_btag_jet1pt
▶ combine_2_jet_1_btag_jet2pt
▶ combine_2_jet_1_btag_jet3pt
▶ e_2_jet_2_btag_MET
▶ e_2_jet_2_btag_lep1pt
▶ e_2_jet_2_btag_lep1pt_None
▶ e_2_jet_2_btag_lep1eta
▶ e_2_jet_2_btag_jet1eta
▶ e_2_jet_2_btag_jet2eta
▶ e_2_jet_2_btag_jet3eta
▶ e_2_jet_2_btag_jet1pt
▶ e_2_jet_2_btag_jet2pt
▶ e_2_jet_2_btag_jet3pt
▶ mu_2_jet_2_btag_MET
▶ mu_2_jet_2_btag_lep1pt
▶ mu_2_jet_2_btag_lep1pt_None
▶ mu_2_jet_2_btag_lep1eta
▶ mu_2_jet_2_btag